# Sección 2. Análisis de sentimientos con redes neuronales recurrentes LSTM utilizando Pytorch. 

In [1]:
# Bibliotecas requeridas
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import re
import spacy
from collections import Counter
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import string
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from sklearn.metrics import mean_squared_error

import matplotlib.pyplot as plt

## Problema

Se desea que, dado un comentario de crítica acerca de la experiencia vivida dentro de un hotel determinado, predecir la calificación correspondiente a ese comentario. La calificación va del 1 al 5 siendo 5 la mejor calificación y 1 la peor calificación. 

## Objetivo
1. Poner en práctica habilidades de investigación y documentación de resultados.
2. Aplicar el conocimiento teórico sobre redes neuronales recurrentes LSTM.
3. Experimentar con el flujo completo de trabajo requerido en proyectos de
aprendizaje automático para realizar análisis de sentimientos a partir de datos en
lenguaje natural.
4. Fortalecer capacidades en los estudiantes en el uso de bibliotecas de aprendizaje
automático como PyTorch y otras.

## Descripción de datos

Los datos vienen dentro de un archivo csv que contiene 2 columnas, la primer columna es el comentario que se le realizó a una hotel determianda y la segunda columna trae la evaluación que se le dió al hotel. La evaluación que se le puede dar al comentario es de un rango del 1 al 5.

Los datos utilizado en el ejemplo estás disponibles en https://www.kaggle.com/code/wiktorbrk/trip-advisor-reviews-sentiment-analysis/data. La descripción del sitio indica "este es un conjunto de datos que contiene un comentario hacia un hotel y una calificación por parte del usuario"

El conjunto de datos consta de las siguientes columnas:

Review: Un texto que es el comentario
Rating: Numero positivo del 1 al 5 que indica la calificación al hotel.

## Carga y preparación de datos

In [2]:
# Leer los datos de ejemplos
reviews = pd.read_csv("hotel_reviews.csv")
#print(reviews.shape)
reviews.head()

,Review,Rating
0,nice hotel expensive parking got good deal sta...,4
1,ok nothing special charge diamond member hilto...,2
2,nice rooms not 4* experience hotel monaco seat...,3
3,"unique, great stay, wonderful time hotel monac...",5
4,"great stay great stay, went seahawk game aweso...",5


In [3]:
#Cambio de la numeración de la clasificaciones de 0 a 4
zero_numbering = {1:0, 2:1, 3:2, 4:3, 5:4}
reviews['Rating'] = reviews['Rating'].apply(lambda x: zero_numbering[x])

## Estadísticas de los datos

In [4]:
# Estadísticas de los datos
pd.set_option('display.float_format', lambda x: '%.2f' % x)
reviews.describe()

,Rating
count,20491.00
mean,2.95
std,1.23
min,0.00
25%,2.00
50%,3.00
75%,4.00
max,4.00


In [5]:
# Verificación de qué tan bien balanceadas están las clases.
Counter(reviews['Rating'])

Counter({3: 6039, 1: 1793, 2: 2184, 4: 9054, 0: 1421})

## Creación definición de modelo y definición de sus datos

In [6]:
# Tokenización: proceso de separar un fragmento de texto en 
#  unidades más pequeñas llamadas tokens. 
#  Los tokens pueden ser palabras, caracteres o sub-palabras.
tok = spacy.blank("en")

def tokenize (text):
    text = re.sub(r"[^\x00-\x7F]+", " ", text)
    regex = re.compile('[' + re.escape(string.punctuation) + '0-9\\r\\t\\n]') # remove punctuation and numbers
    nopunct = regex.sub(" ", text.lower())
    return [token.text for token in tok.tokenizer(nopunct)]

In [7]:
# Se cuenta la cantidad de ocurrencias de cada token 
# en el corpus.

#count number of occurences of each word
counts = Counter()
for index, row in reviews.iterrows():
    counts.update(tokenize(row['Review']))

In [8]:
# Se crea el vocabulario
vocab2index = {"":0, "UNK":1}
words = ["", "UNK"]
for word in counts:
    vocab2index[word] = len(words)
    words.append(word)

#print(vocab2index)  

In [9]:
def encode_sentence(text, vocab2index, N=70):
    """
    Codificación de una oración antes de ser utilizada por el modelo. 
    Parámetros:
       text: el texto a procesar
       vocab2index: diccionario con el vocabulario a utilizar. 
       N: largo máximo
    """
    tokenized = tokenize(text)
    encoded = np.zeros(N, dtype=int)
    
    # El get en diccionario permite definir un valor si un item no existe ("UNK").  
    enc1 = np.array([vocab2index.get(word, vocab2index["UNK"]) for word in tokenized])
    
    # Largo máximo del resultado.
    length = min(N, len(enc1))
    encoded[:length] = enc1[:length]
    return encoded, length

reviews['encoded'] = reviews['Review'].apply(lambda x: np.array(encode_sentence(x,vocab2index ), dtype=object))
#print(reviews.head())

In [10]:
# Extracción de características y target.
X = list(reviews['encoded'])
y = list(reviews['Rating'])

# División de datos de entrenamiento y validación
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2)

In [11]:
# Definción de la clase Dataset para manejo de los datos
class ReviewsDataset(Dataset):
    def __init__(self, X, Y):
        self.X = X
        self.y = Y
        
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        return torch.from_numpy(self.X[idx][0].astype(np.int32)), self.y[idx], self.X[idx][1]

In [47]:
class LSTM_fixed_len(torch.nn.Module) :
    """
    Clase para realizar la clasificación de las oraciones. 
    """
    def __init__(self, vocab_size, embedding_dim, hidden_dim, tagset_size=5) :
        """
        Inicialización de la clase.
        Parámetros:
           embedding_dim: dimesionalidad del vector de palabras. 
           hidden_dim: dimensión de la capa oculta de la red. 
           vocab_size: tamaño del vocabulario.  
           tagset_size: número de clases.
        """
        super().__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.linear = nn.Linear(hidden_dim, tagset_size)
        
        # Durante el entrenamiento, pone a cero aleatoriamente algunos de los elementos 
        # del tensor de entrada con probabilidad p utilizando muestras de una 
        # distribución de Bernoulli. Esta ha demostrado ser una técnica eficaz para la regularización.
        self.dropout = nn.Dropout(0.2)
        
    def forward(self, x, l):
        x = self.embeddings(x)
        x = self.dropout(x)
        lstm_out, (ht, ct) = self.lstm(x)
        return self.linear(ht[-1])

In [48]:
vocab_size = len(words)

EMBEDDING_DIM = 6
HIDDEN_DIM = 6

# Instancia del modelo
model_fixed = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, 50, 50)
#model_fixed =  LSTM_fixed_len(vocab_size, 50, 50)

## Definición de los hiper-parámetros

In [49]:
import torch.nn.functional as F

#Funcion de loss
loss_function = F.cross_entropy

#Funcipon de optimización
parameters = filter(lambda p: p.requires_grad, model_fixed.parameters())
optimizer = torch.optim.SGD(parameters, lr=0.001, momentum=0.9)

## Datos de pruebas y validación

In [50]:
# Creación de los datasets de entrenamiento y validación

batch_size = 5000

train_ds = ReviewsDataset(X_train, y_train)
valid_ds = ReviewsDataset(X_valid, y_valid)

#Datos de entrenamiento y validación
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
val_dl = DataLoader(valid_ds, batch_size=batch_size)

In [51]:
#guardo para mostar la gráfica de error
acc_l = []
loss_l = []
val_acc_l = []
val_loss_l = []

In [52]:
#Esta funcion me ayuda a extraer datos para poder graficar la curva de error
def validation_metrics (model, valid_dl):
    model.eval()
    correct = 0
    total = 0
    sum_loss = 0.0
    sum_rmse = 0.0
    for x, y, l in valid_dl:
        x = x.long()
        y = y.long()
        y_hat = model(x, l)
        loss = F.cross_entropy(y_hat, y)
        pred = torch.max(y_hat, 1)[1]
        correct += (pred == y).float().sum()
        total += y.shape[0]
        sum_loss += loss.item()*y.shape[0]
        sum_rmse += np.sqrt(mean_squared_error(pred, y.unsqueeze(-1)))*y.shape[0]
    return sum_loss/total, correct/total, sum_rmse/total

## Entrenar el modelo

In [53]:
def train_model(model,train_dl,epochs):
    """
    Entrenamiento del modelo utilizando PyTorch.
    """
    res = []
    #optimizer = torch.optim.Adam(parameters, lr=lr)
    for i in range(epochs):
        model.train()
        sum_loss = 0.0
        total = 0
        for x, y, l in train_dl:
            x = x.long()
            y = y.long()
            y_pred = model(x, l)
            optimizer.zero_grad()
            loss = loss_function(y_pred, y)
            loss.backward()
            optimizer.step()
            sum_loss += loss.item()*y.shape[0]
            #loss_values.append(loss_l)
            total += y.shape[0]
            
            res=y       
        
        val_loss, val_acc, val_rmse = validation_metrics(model, val_dl)
        
        #guardo datos para graficarlos en curva de error
        val_loss_l.append(val_loss)
        val_acc_l.append(val_acc)
        loss_l.append(sum_loss/total)
        acc_l.append(val_rmse)
        
        if i % 5 == 1:
            print("train loss %.3f, val loss %.3f, val accuracy %.3f, and val rmse %.3f" % (sum_loss/total, val_loss, val_acc, val_rmse))
    print("Termina el entrenamiento")
    return res

In [54]:
prediccion = train_model(model_fixed,train_dl, epochs=30)

TypeError: forward() takes 2 positional arguments but 3 were given

## Gráfica de curva de error

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline 

epochs = range(len(acc_l))

loss = loss_l
val_loss = val_loss_l

plt.plot(epochs, loss, 'r', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.ylabel('loss')  
plt.xlabel('epoch')
plt.legend()
plt.show()

## Evaluación del modelo - Acurrancy

In [ ]:
# Validación del modelo 
average_loss, accuracy, average_rmse = validation_metrics (model_fixed, val_dl)

print ("Exactitud: ", accuracy * 100, "%")

## Evaluación del modelo - Matriz de confusión

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

#y_pred = model_fixed(a,c)
#una lista con calificaciones del 1 al 5

y_true = y_train[15000:] #los últimos 1392 son los resultados correctos de los targets
y_pred = prediccion

#y_pred = model(x, l)
#y_pred = y_train

cm = confusion_matrix(y_true, y_pred)

plt.figure(figsize = (5,5))
sns.heatmap(cm,cmap= "Blues", 
            linecolor = 'black', 
            linewidth = 1, 
            annot = True, 
            fmt='', 
            xticklabels = ['1','2','3','4','5'], 
            yticklabels = ['1','2','3','4','5'])
plt.xlabel("Predicted")
plt.ylabel("Actual")

Observando la matriz de confusión se puede deducir que el modelo no está muy bien entrenado, ya que se confunde entre clases. Otra observación es que aunque el modelo no esté muy bien entrenado, predice correctamente la mayoría de manera muy buena.

## Mejoras al modelo - Balanceo de carga (Clase con menor datos)

In [ ]:
#guardo para mostar la gráfica de error
acc_l = []
loss_l = []
val_acc_l = []
val_loss_l = []

In [ ]:
#Balanceo las clases del csv utilizando la clase que tiene menos datos

df = pd.read_csv("hotel_reviews.csv")

a = pd.DataFrame(df[df.Rating == 1][:1420])
b = pd.DataFrame(df[df.Rating == 2][:1420])
c = pd.DataFrame(df[df.Rating == 3][:1420])
d = pd.DataFrame(df[df.Rating == 4][:1420])
e = pd.DataFrame(df[df.Rating == 5][:1420])

frames = [a,b,c,d,e]
result = pd.concat(frames)

#Cambio de la numeración de la clasificaciones de 0 a 4
zero_numbering = {1:0, 2:1, 3:2, 4:3, 5:4}
result['Rating'] = result['Rating'].apply(lambda x: zero_numbering[x])

# Verificación de qué tan bien balanceadas están las clases.
Counter(result['Rating'])

In [ ]:
result['encoded'] = result['Review'].apply(lambda x: np.array(encode_sentence(x,vocab2index ), dtype=object))
# Extracción de características y target.
X2 = list(result['encoded'])
y2 = list(result['Rating'])

# División de datos de entrenamiento y validación
X_train2, X_valid2, y_train2, y_valid2 = train_test_split(X2, y2, test_size=0.5)

In [ ]:
# Creación de los datasets de entrenamiento y validación

batch_size = 5000

train_ds2 = ReviewsDataset(X_train2, y_train2)
valid_ds2 = ReviewsDataset(X_valid2, y_valid2)

#Datos de entrenamiento y validación
train_dl2 = DataLoader(train_ds2, batch_size=batch_size, shuffle=True)
val_dl2 = DataLoader(valid_ds2, batch_size=batch_size)

In [ ]:
model_fixed2 =  LSTM_fixed_len(len(words),50,50)

In [ ]:
prediccion2 = train_model(model_fixed2,train_dl2, epochs=30)

In [ ]:
# Validación del modelo 
average_loss, accuracy2, average_rmse = validation_metrics (model_fixed2, val_dl2)
print ("Exactitud: ", accuracy * 100, "%")

Al principio nuestro modelo nos daba un rendimiento bajo, aunque con esta "mejora" nos haya dado parecido, el entrenamiento lo ha realizado en menos tiempo y con un resultado favorable.

## Mejoras al modelo - (Aumento de epochs)

In [ ]:
prediccion = train_model(model_fixed,train_dl, epochs=30)


In [ ]:
# Validación del modelo 
average_loss, accuracy, average_rmse = validation_metrics (model_fixed, val_dl)
print ("Exactitud: ", accuracy * 100, "%")

## Conclusiones

- La exactitud indica que el modelo no logra buenos resultados, sin embargo, el RMSE que es la desviación estándar de los residuos (errores de predicción), que en otras palabras, representa qué tan concentrados están los datos alrededor de la línea de mejor ajuste, está alrededor de 1 punto lo que indica que las predicciones no son tan malas.

- Las clases no están desbalanceadas como se muestra al principio. Se podría variar la composición por ejemplo fusionando algunas y evaluar si eso mejora el rendimiento.

- Antes de clasificar textos en una red LSTM se debe revisar el texto ya que puede contener datos innecesarios, osea, realizar un preprocesamiento eliminando texto innecesario.

- Se deben de configurar correctamente los parámetros de la función de entrenamiento y de la red, esto ya que un debalance de estos produce retardos o mal entrenamiento.

## Referencias
- Sentiment Analysis of IMDB Movie Reviews. Kaggle.com. (2022). Retrieved 18 May 2022, from https://www.kaggle.com/code/lakshmi25npathi/sentiment-analysis-of-imdb-movie-reviews/data.
- Sentiment Analysis of IMDB Movie Reviews. Kaggle.com. (2022). Retrieved 18 May 2022, from https://www.kaggle.com/code/lakshmi25npathi/sentiment-analysis-of-imdb-movie-reviews/notebook.
- animal-image-classifications/I_notebook.ipynb at master · imamun93/animal-image-classifications. GitHub. (2022). Retrieved 18 May 2022, from https://github.com/imamun93/animal-image-classifications/blob/master/I_notebook.ipynb.